# ConversationEntityMemory

엔티티 메모리는 대화에서 특정 엔티티에 대한 주어진 사실을 기억합니다.

엔티티 메모리는 엔티티에 대한 정보를 추출하고(LLM 사용) 시간이 지남에 따라 해당 엔티티에 대한 지식을 축적합니다(역시 LLM 사용).


In [17]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [18]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

##### Entity 메모리를 효과적으로 사용하기 위하여, 제공되는 프롬프트를 사용합니다.


당신은 OpenAI가 훈련한 대규모 언어 모델을 기반으로 한 인간의 보조자입니다.

당신은 단순한 질문에 답하는 것부터, 폭넓은 주제에 대한 심도 있는 설명과 토론에 이르기까지 다양한 작업을 도울 수 있도록 설계되었습니다. 언어 모델로서, 입력받은 내용을 바탕으로 사람과 유사한 텍스트를 생성할 수 있으며, 이를 통해 자연스러운 대화를 나누고 주제에 맞는 일관된 답변을 제공할 수 있습니다.

또한 당신은 끊임없이 학습하고 개선되며, 능력도 계속 발전합니다. 많은 양의 텍스트를 처리하고 이해할 수 있으며, 이를 활용해 다양한 질문에 대해 정확하고 유익한 답변을 줄 수 있습니다. 인간이 제공한 개인화된 정보에 일부 접근할 수 있으며, 입력받은 내용을 기반으로 새로운 텍스트를 생성해 토론에 참여하거나 설명과 묘사를 제공할 수도 있습니다.

결론적으로, 당신은 폭넓은 작업을 지원하고 다양한 주제에 대한 귀중한 통찰과 정보를 제공할 수 있는 강력한 도구입니다. 인간이 특정 질문에 대한 도움이 필요하든, 단순히 어떤 주제에 대해 대화를 나누고 싶든 간에, 당신은 이를 돕기 위해 존재합니다.

In [19]:
# Entity Memory를 사용하는 프롬프트 내용을 출력합니다.
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [20]:
# LLM 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# ConversationChain 을 생성합니다.
conversation = ConversationChain(
    llm=llm,
    # 대화 중 등장하는 엔티티(사람, 장소, 사물 등)를 정리·업데이트 하도록 모델에 지시
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
)

대화를 시작합니다.

입력한 대화를 바탕으로 `ConversationEntityMemory` 는 주요 Entity 정보를 별도로 저장합니다.


In [21]:
conversation.predict(
    input="짱구와 철수는 같은 떡잎유치원에 다니는 친구이다."
    "짱구는 못말리고 철수는 똑똑하다."
    "둘은 틱틱대지만 사이가 좋다."
)

'짱구와 철수는 서로 성격이 다르지만, 같은 떡잎유치원에 다니며 좋은 친구 사이인 것 같아요. 짱구는 활발하고 장난꾸러기인 반면, 철수는 똑똑하고 차분한 모습이 서로를 잘 보완해 주는 것 같네요. 가끔 티격태격해도 결국에는 서로를 이해하고 아끼는 모습이 참 보기 좋아요! 혹시 이 두 친구에 대해 더 이야기해 보고 싶나요?'

Entity는 `memory.entity_store.store` 에서 확인할 수 있습니다.


##### 짱구와 철수, 떡잎유치원에 대한 정보를 각각 정리했으나 혹시 openAI가 짱구는 못말려에 대해 알 가능성이 있어서 추가로 답한것 같다.

In [22]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store

{'짱구': '짱구는 떡잎유치원에 다니는 친구로, 못말리고 활발한 성격을 가지고 있다.',
 '철수': '철수는 똑똑하고 짱구와 함께 같은 떡잎유치원에 다니는 친구이며, 가끔 티격태격하지만 사이가 좋은 편이다.',
 '떡잎유치원': '떡잎유치원은 짱구와 철수가 다니는 유치원이다.'}

In [23]:
# LLM 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# ConversationChain 을 생성합니다.
conversation = ConversationChain(
    llm=llm,
    # 대화 중 등장하는 엔티티(사람, 장소, 사물 등)를 정리·업데이트 하도록 모델에 지시
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
)

In [24]:
conversation.predict(
    input=(
        "민준과 수아는 같은 대학교를 졸업했습니다. "
        "민준은 현재 구글에서 데이터 엔지니어로 일하고 있고, "
        "수아는 카카오에서 마케팅 매니저로 근무 중입니다.\n\n"

        "둘은 대학 시절 동아리에서 처음 만나 친해졌으며, "
        "졸업 후에도 정기적으로 연락을 주고받으며 지냈습니다. "
        "최근에는 각자의 회사에서 겪은 경험을 바탕으로, "
        "스타트업을 함께 창업할 계획을 세우고 있습니다.\n\n"

        "그들이 준비하는 회사는 인공지능을 활용한 헬스케어 서비스 기업입니다. "
        "민준은 데이터와 시스템 구축을 담당하고, "
        "수아는 브랜드 전략과 마케팅을 총괄할 예정입니다. "
        "현재는 시드 투자자를 찾는 단계에 있습니다."
    )
)


'민준과 수아가 함께 창업하는 인공지능 헬스케어 스타트업은 매우 유망해 보입니다. 민준의 데이터 엔지니어링 역량과 수아의 마케팅 경험이 잘 조화를 이루어 성공 가능성이 높을 것 같네요. \n\n혹시 시드 투자자에게 어필할 수 있는 사업 계획서나 피치덱 작성에 도움이 필요하시거나, 헬스케어 AI 시장 동향, 경쟁사 분석, 혹은 마케팅 전략 수립에 관한 조언이 필요하시면 언제든 말씀해 주세요. 창업 준비 과정에서 필요한 자료나 아이디어도 함께 고민해 드릴 수 있습니다!'

In [25]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store

{'민준': '민준은 구글에서 데이터 엔지니어로 일하며, 수아와 함께 인공지능 헬스케어 스타트업을 창업할 계획으로 데이터와 시스템 구축을 담당하고 있다.',
 '수아': '수아는 카카오에서 마케팅 매니저로 근무하며, 민준과 함께 인공지능 헬스케어 스타트업을 창업해 브랜드 전략과 마케팅을 총괄할 예정이다.',
 '구글': '구글은 민준이 데이터 엔지니어로 근무하고 있는 회사이다.',
 '카카오': '카카오는 수아가 마케팅 매니저로 근무 중인 회사이다.'}